In [18]:
import sys
sys.path.append('../common')
import utils
import importlib
importlib.reload(utils)
from utils import IOHandler as IO

import time
import random
import sys

from PIL import Image
import numpy as np

import pynput.keyboard as kb

from datetime import datetime
from multiprocessing import Pool
from functools import partial
from enum import Enum

global possible
global all_scores
global unused

io = utils.IOHandler(offset=(1081, 675), game_dims=(1200,900), verbose=True)

In [19]:

# verbose = len(sys.argv) > 1 and sys.argv[1] == "-v"
verbose = True

MAX_DIGITS = 6
MAX_CHANCES = 15
# TODO: use load_images_from_directory instead
level_masks = []
for i in range(31):
    img = Image.open('levels/mask_' + str(i+1) + ".png")
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    level_masks.append(arr)

# blank (10), none (11)
digit_masks = []
for i in range(12):
    img = Image.open('digits/mask_' + str(i) + ".png")
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    digit_masks.append(arr)

# green (0), red (1), blank (2)
ball_masks = []
for i in range(3):
    img = Image.open('balls/mask_' + str(i) + ".png")
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    ball_masks.append(arr)

time_masks = []
for i in range(10):
    img = Image.open('time/mask_' + str(i) + "_o.png")
    arr = np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]
    time_masks.append(arr)

time_blank_mask = np.array(Image.open('time/mask_blank.png')).swapaxes(0, 1).astype(np.int32)[:,:,:3]

In [20]:
def debug(*args, **kwargs):
    if verbose:
        print(*args, **kwargs)
        

class Screen(Enum):
    HOME = 'home'
    GAME_OVER = 'game_over'
    END_GAME = 'end_game'
    NEW_LEVEL = 'new_level'
    GREAT_JOB = 'great_job'
    ARCADE = 'arcade'
    MIDGAME = 'midgame'
    EASY_MODE = 'easy_mode'
    GREAT_PLAY = "great_play"

SCREEN_TYPES = {
    Screen.HOME: [1066, 684, [229, 99, 42]], # orange gumball
    Screen.GAME_OVER: [680, 380, [246, 196, 195]], # piggy bank
    Screen.END_GAME: [420, 860, [247, 207, 78]], # "end game" button, either ran out of guesses or time
    Screen.NEW_LEVEL: [850, 460, [245, 199, 69]], # "Start!", "Level: X", or congrats pop-up
    Screen.GREAT_JOB: [230, 664, [3, 14, 147]], # "great job!" pop-up
    Screen.ARCADE: [170, 852, [145, 61, 148]], # magenta arcade cabinet
    Screen.MIDGAME: [1095, 60, [92, 29, 196]], # purple music note
    Screen.EASY_MODE: [485, 44, [208, 126, 48]], # orange outline in level area
    Screen.GREAT_PLAY: [880, 600, [122, 26, 19]], # red gift box
}

io.set_screen_types(SCREEN_TYPES)


LEVEL_DIGITS = {
    'normal': [3, 3, 3, 4, 4, 4, 5, 5, 5, 6], 
    'easy': [3, 3, 3, 3, 4]
}
LEVEL_CHANCES = {
    'normal': [12,11,10]*3 + [11] + [11,10,9]*3 + [9] + [9,8,7]*3 + [8],
    'easy': [15]*5,
}
LEVEL_FIRST_GUESS = ['012', '0123', '01234', '012345', '012', '0123', '00123', '001123']
LEVEL_TIME = [120,140,160,190,210,230,260,280,300,330,90,110,130,160,180,200,230,250,270,300,60,80,100,130,150,170,200,220,240,270]


def get_level_digits(level, difficulty):
    return LEVEL_DIGITS[difficulty][(level-1) % 10]

def get_level_chances(level, difficulty):
    return LEVEL_CHANCES[difficulty][level-1]

def get_level_has_repeats(level):
    return level > 10

def get_level_time(level):
    return LEVEL_TIME[level-1]

def get_best_first_guess(num_digits, has_repeats):
    if has_repeats:
        return LEVEL_FIRST_GUESS[num_digits + 1]
    return LEVEL_FIRST_GUESS[num_digits - 3]


def parse_level():
    if io.is_screen_type(Screen.EASY_MODE):
        level_img = io.capture_portion(615, 23, 45, 45)
    else:
        level_img = io.capture_portion(454, 23, 45, 45)
    curr_level = str(IO.get_best_mask(level_img, level_masks[:-1])[0] + 1)
    return curr_level

def parse_time():
    if io.is_screen_type(Screen.EASY_MODE):
        return -1
    time_img = io.capture_portion(751, 30, 68, 30)
    digit_minutes = IO.get_best_mask(time_img[:20, :30], time_masks)[0]
    digit_tens = IO.get_best_mask(time_img[28:28+20, :30], time_masks)[0]
    digit_ones = IO.get_best_mask(time_img[48:48+20, :30], time_masks)[0]
    return digit_minutes*60 + digit_tens*10 + digit_ones


def parse_digit(i, j):
    x0, y0 = (687,164)
    sx, sy = (32, 44)
    w, h = (30, 30)

    x = x0 + sx * j
    y = y0 + sy * i

    digit_img = io.capture_portion(x, y, w, h)

    idx = IO.get_best_mask(digit_img, digit_masks)[0]
    if idx == 10:
        digit = '-'
    elif idx == 11:
        digit = ''
    else:
        digit = str(idx)

    return digit

def parse_attempt(i):
    attempt = ''
    for j in range(MAX_DIGITS):
        attempt += parse_digit(i, j)
    return attempt

def parse_attempts():
    attempts = []
    for i in range (MAX_CHANCES):
        attempts.append(parse_attempt(i))
    return attempts

def capture_response_img(i):
    x0, y0 = (918, 165)
    w, h = (184, 13)
    sy = 44
    x = x0
    y = y0 + i*sy
    response_img = io.capture_portion(x, y, w, h)
    return response_img

def parse_response(i):
    w, h = (22, 13)
    sx = 30
    response_img = capture_response_img(i)
    response = [0, 0]
    for j in range (MAX_DIGITS):
        x = sx * j
        ball_img = response_img[x:x+w,:h, :]
        color = IO.get_best_mask(ball_img, ball_masks)[0]
        if color == 0:
            response[0] += 1
        elif color == 1:
            response[1] += 1
    return response

def parse_responses():
    return [parse_response(i) for i in range(MAX_CHANCES)]


def compare_codes(a, b):
    """Return # of red & green numbers"""
    green = 0
    count1 = [0] * 10
    count2 = [0] * 10
    for dig1, dig2 in zip(a, b):
        if dig1 == dig2:
            green += 1
        else:
            count1[int(dig1)] += 1
            count2[int(dig2)] += 1
    red = sum(map(min, count1, count2))
    return int(green), int(red)

def unique_chars(s):
    return len(set(s)) == len(s)

def calc_worst_elims(word, possible, all_scores):
    worst_elims = len(possible)
    for score in all_scores:
        these_possible = [t for t in possible if compare_codes(t, word) == score]
        these_elims = len(possible) - len(these_possible)
        worst_elims = min(worst_elims, these_elims)
    return worst_elims

# TODO: move break/kill program to utils and make names/keys consistent in bots
def on_press(key):
    global break_program, listener
    if key == kb.Key.esc:
        print("Escape pressed to quit")
        listener.stop()
        break_program = True
    elif key == kb.Key.tab:
        print("Tab pressed to take screenshot")
        io.save_game_capture()

# Get into the main game
# TODO: move to utils?
def get_into_game(difficulty='normal'):
    global break_program
    while not break_program:
        screen_types = io.calc_screen_types()
        print(screen_types)
        
        if Screen.ARCADE in screen_types:
            io.click_mouse(970, 760) # "PLAY GAME" button
        elif Screen.GREAT_PLAY in screen_types:
            io.click_mouse(590, 1000) # "Collect" button
        elif Screen.GAME_OVER in screen_types:
            io.click_mouse(220, 703) # Replay button
        elif Screen.HOME in screen_types:
            debug(f"Start game - difficulty {difficulty}")
            if difficulty == 'easy':
                io.click_mouse(450, 745) # easy button
            else:
                io.click_mouse(690, 745) # normal button
            io.click_mouse(185, 845) # 1 player game button
        elif Screen.END_GAME in screen_types:
            io.click_mouse(320, 860) # End Game button
        elif Screen.NEW_LEVEL in screen_types:
            pass
        elif Screen.GREAT_JOB in screen_types:
            pass
        elif Screen.MIDGAME in screen_types:
            return

        time.sleep(0.5)


In [21]:
game_start = None

break_program = False
listener = kb.Listener(on_press=on_press)
listener.start()

# TODO: faster suggestion finding!
# TODO; could make a seperate environment that doesn't have webkinz obstacles, direct interface
# TODO: better time distribution for suggestion finding!
# TODO: allow exiting
# TODO: could run out of time while typing

# TODO: simplify logic by using get_into_game() and fewer while loops

level = 0

io.click_mouse(0, 0)
desired_difficulty = 'normal'
get_into_game(desired_difficulty)

break_program = False

game_start = datetime.now()
while not break_program:
    time.sleep(0.1)
    num_attempts = 0

    level = int(parse_level())
    is_easy_mode = io.is_screen_type(Screen.EASY_MODE)
    difficulty = 'easy' if is_easy_mode else 'normal'
    num_digits = get_level_digits(level, difficulty)
    chances = get_level_chances(level, difficulty)
    has_repeats = get_level_has_repeats(level)

    print(f"---LEVEL {level}: ({num_digits} digits, {'has' if has_repeats else 'no'} repeats, difficuly {difficulty})---")

    # init possible, unused, and response codes
    possible = [str(i).zfill(num_digits) for i in range(10**(num_digits))]
    random.shuffle(possible)
    unused = possible
    if not has_repeats:
        possible = [code for code in possible if unique_chars(code)]
    
    all_scores = [(r, g) for r in range(num_digits + 1) for g in range(num_digits - r + 1)]
    # TODO: should we be removing these?
    all_scores.remove((num_digits-1, 1))
    all_scores.remove((num_digits, 0))

    best_worst_elims = -1

    while not break_program:
        time.sleep(0.1)
        debug("possible:\t", len(possible))
        
        attempt = parse_attempt(num_attempts)
        if '-' in attempt: 
            if num_attempts == 0:
                suggestion = get_best_first_guess(num_digits, has_repeats)
            else:
                # Find guess with best worst case scenario
                # Check time remaining so we don't take it all
                time_remaining = parse_time()
                search_start = time.time()
                if time_remaining > 0:
                    # max_search_time = time_remaining / (chances - num_attempts + 1) - 4
                    max_search_time = (time_remaining / 3) - 3
                    # max_search_time = 0
                    # max_search_time = time_remaining / 2 - 3
                else:
                    max_search_time = 60
                
                search_set = possible
                if num_digits < 5:                        
                    # unused = sorted(unused, key=lambda x: x in possible, reverse=True) # NOTE EXPENSIVE
                    # search_set = unused
                    max_search_time = (time_remaining / 3) - 3
                # else:
                #     search_set = possible

                max_search_time = 5
                debug(f"Searching ({max_search_time:.1f}s limit)...")
                results = []
                best_worst_elims = 0
                suggestion = search_set[0] # TODO: this throws at end of game
                for word in search_set:
                    result = calc_worst_elims(word, possible, all_scores)
                    results.append(result)
                    if time.time() - search_start >= max_search_time:
                        debug("INFO: ran out of time to keep searching")
                        break
                    # print(f'Still searching... {result}')

                # TODO: fix Pool not working in ipynb
                # with Pool(8) as p:
                #     for result in p.imap(partial(calc_worst_elims, possible=possible, all_scores=all_scores), search_set, chunksize=1):
                #         results.append(result)
                #         if time.time() - search_start >= max_search_time:
                #             debug("INFO: ran out of time to keep searching")
                #             p.terminate()
                #             break
                #         print(f'Still searching... {result}')

                debug(f"searched {len(results)/len(search_set)*100:.1f}% ({len(results)}/{len(search_set)}) guesses in {time.time() - search_start:.2f}s")
                best_worst_elims = max(results)
                max_idx = results.index(best_worst_elims)
                suggestion = search_set[max_idx]
                debug(f"min elims: {best_worst_elims}/{len(possible)}")

            print ("#{}:\t".format(num_attempts+1), suggestion)

            # Type in suggestion
            attempt = parse_attempt(num_attempts)
            while not break_program and (len(attempt) != num_digits or '-' in attempt) and not io.is_screen_type(Screen.GREAT_JOB):   
                time.sleep(0.1)
                correct_digits = 0
                for i, c in enumerate(attempt):
                    if attempt[i] == suggestion[i]:
                        correct_digits += 1
                    else:
                        break
                for i in range(len(attempt)-1, correct_digits-1, -1):
                    while not break_program and attempt[i] != '-':
                        time.sleep(0.1)
                        debug("press backspace")
                        io.click_key(kb.Key.backspace)
                        attempt = parse_attempt(num_attempts)
                for i in range(correct_digits, len(attempt)):
                    while not break_program and attempt[i] != suggestion[i]:  
                        time.sleep(0.1)
                        print(attempt, suggestion)
                        debug("press {}".format(suggestion[i]))
                        io.click_key(suggestion[i])
                        attempt = parse_attempt(num_attempts)

        num_attempts += 1
        if attempt in unused:
            unused.remove(attempt)

        debug(attempt)
        debug("num_attempts:\t", num_attempts)

        # if finished level, wait for & go to the next one
        if io.is_screen_type(Screen.GREAT_JOB):
            # io.save_game_capture()
            while not break_program and io.is_screen_type(Screen.GREAT_JOB):
                time.sleep(0.1)
                pass
            while not break_program and parse_level() != str(level + 1):
                time.sleep(0.1)
                pass
            while not break_program and io.is_screen_type(Screen.NEW_LEVEL):
                time.sleep(0.1)
                pass
            break

        # TODO: play again if game over, check for fail or level 30
        # if failed level, go to game over screen
        if io.is_screen_type(Screen.END_GAME):
            # io.save_game_capture()
            while not break_program and not io.is_screen_type(Screen.GAME_OVER):
                time.sleep(0.1)
                io.click_mouse(317, 864)
                pass
        
        # wait for response
        debug("response:\t", end='')
        while not break_program and parse_attempt(num_attempts) == '-'*num_digits:
            time.sleep(0.1)
            io.click_key("0")
        response = parse_response(num_attempts-1)
        debug(response)

        if int(response[0]) == num_digits:
            debug("attempts:\t", num_attempts)
            break   # Move onto next level
        else:
            score = int(response[0]), int(response[1])
            difference = len(possible)
            possible = [p for p in possible if compare_codes(p, attempt) == score]
            difference -= len(possible)
            if attempt in possible:
                possible.remove(attempt)
            if attempt in unused:
                unused.remove(attempt)
            if difference < best_worst_elims:
                print(f"ERROR: {best_worst_elims} > {difference}")

{<Screen.GREAT_PLAY: 'great_play'>}
{<Screen.GAME_OVER: 'game_over'>}
{<Screen.HOME: 'home'>}
Start game - difficulty normal
{<Screen.MIDGAME: 'midgame'>, <Screen.NEW_LEVEL: 'new_level'>}
{<Screen.MIDGAME: 'midgame'>, <Screen.NEW_LEVEL: 'new_level'>}
{<Screen.MIDGAME: 'midgame'>, <Screen.NEW_LEVEL: 'new_level'>}
{<Screen.MIDGAME: 'midgame'>, <Screen.NEW_LEVEL: 'new_level'>}
{<Screen.MIDGAME: 'midgame'>, <Screen.NEW_LEVEL: 'new_level'>}
{<Screen.MIDGAME: 'midgame'>}
---LEVEL 1: (3 digits, no repeats, difficuly normal)---
possible:	 720
#1:	 012
--- 012
press 0
0-- 012
press 1
01- 012
press 2
012
num_attempts:	 1
response:	[0, 1]
possible:	 252
Searching (5.0s limit)...
searched 100.0% (252/252) guesses in 1.56s
min elims: 172/252
#2:	 134
press backspace
--- 134
press 1
1-- 134
press 3
13- 134
press 4
134
num_attempts:	 2
response:	[0, 0]
possible:	 80
Searching (5.0s limit)...
searched 100.0% (80/80) guesses in 0.18s
min elims: 59/80
#3:	 726
press backspace
--- 726
press 7
7-- 726
pre

IndexError: list index out of range

Escape pressed to quitEscape pressed to quit

